In [6]:
from jsonpath_ng import jsonpath, parse

jsonpath_expr = parse('foo[*].baz')

[match.value for match in jsonpath_expr.find({'foo': [{'baz': 1}, {'baz': 2}]})]
[1, 2]

[str(match.full_path) for match in jsonpath_expr.find({'foo': [{'baz': 1}, {'baz': 2}]})]
['foo.[0].baz', 'foo.[1].baz']

jsonpath.auto_id_field = 'id'
[match.value for match in parse('foo[*].id').find({'foo': [{'id': 'bizzle'}, {'baz': 3}]})]

[match.value for match in parse('a.*.b.`parent`.c').find({'a': {'x': {'b': 1, 'c': 'number one'}, 'y': {'b': 2, 'c': 'number two'}}})]
['number two', 'number one']

['number two', 'number one']

In [14]:
from jsonpath_ng.jsonpath import Fields
from jsonpath_ng.jsonpath import Slice

jsonpath_expr_direct = Fields('foo').child(Slice()).child(Fields('baz'))  # This is equivalent


from jsonpath_ng.ext import parse
jsonpath_expr = parse('foo[*].baz')

jsonpath_expr_direct, jsonpath_expr

(Child(Child(Fields('foo'), Slice(start=None,end=None,step=None)), Fields('baz')),
 Child(Child(Fields('foo'), Slice(start=None,end=None,step=None)), Fields('baz')))

In [44]:
import json
from jsonpath_ng.ext import parse

json_payload = {
    "autorizacao": {
        "anoCalendario": "2021",
        "avisoLegal": "Lorem Ipsum Dolor Sit Amet",
        "cnpjTitular": "1234567891011",
        "dataHoraRegistro": "2022-12-28T23:56:02.65",
        "destinatario": "123456789",
        "titular": "abcdefghij",
        "token": "lorem ipsum dolor sit amet"
    },
    "dadosCnpj": [
        {
            "codigo": "5.1",
            "texto": "Data de início de atividades",
            "valor": "20191013"
        },
        {
            "codigo": "5.2",
            "texto": "Capital Social",
            "valor": "100000.00"
        },
        {
            "codigo": "5.3",
            "texto": "Data de exclusão do simples",
            "valor": "null"
        }
    ],
    "dadosAninhados": [
        {
            "tipo": "tipo 1",
            "subtipos": [
                {
                    "subtipo": "subtipo A",
                    "campo": "campo_1A",
                    "valor": "valor_1A"
                },
                {
                    "subtipo": "subtipo B",
                    "campo": "campo_1B",
                    "valor": "valor_1B"
                }
            ]
        },
        {
            "tipo": "tipo 2",
            "subtipos": [
                {
                    "subtipo": "subtipo A",
                    "campo": "campo_2A",
                    "valor": "valor_2A"
                },
                {
                    "subtipo": "subtipo B",
                    "campo": "campo_2B",
                    "valor": "valor_2B"
                }
            ]
        }
    ],
    "campo_lista_dinamica_simples": ["a", "b", "c"],
    "campo_lista_dinamica_complexa": [
        {
            "nome": "fulano",
            "sobrenome": "silva"
        },
        {
            "nome": "fulana",
            "sobrenome": "santos"
        }
    ]
}

lookup_dict = {
    'autorizacao.anoCalendario' : '$.autorizacao.anoCalendario',
    'autorizacao.avisoLegal' : '$.autorizacao.avisoLegal',
    'autorizacao.cnpjTitular' : '$.autorizacao.cnpjTitular',
    'autorizacao.dataHoraRegistro' : '$.autorizacao.dataHoraRegistro',
    'autorizacao.destinatario' : '$.autorizacao.destinatario',
    'autorizacao.titular' : '$.autorizacao.titular',
    'autorizacao.token' : '$.autorizacao.token',
    # 'dadosCnpj.data_de_início_de_atividades' : '$.dadosCnpj[?(@.texto=="Data de início de atividades")].valor',
    # 'dadosCnpj.capital_social' : '$.dadosCnpj[?(@.texto=="Capital Social")].valor',
    # 'dadosCnpj.data_de_exclusão_do_simples' : '$.dadosCnpj[?(@.texto=="Data de exclusão do simples")].valor',
    'dadosCnpj.data_de_início_de_atividades' : '$.dadosCnpj[?(@.codigo=="5.1")].valor',
    'dadosCnpj.capital_social' : '$.dadosCnpj[?(@.codigo=="5.2")].valor',
    'dadosCnpj.data_de_exclusão_do_simples' : '$.dadosCnpj[?(@.codigo=="5.3")].valor',
    'dadosCnpj.data_de_exclusão_do_simples' : '$.dadosCnpj[?(@.codigo=="5.4")].valor',
    'dadosAninhados.tipo_1.subtipo_A' : '$.dadosAninhados[?(@.tipo=="tipo 1")].subtipos[?(@.subtipo=="subtipo A")].valor',
    'dadosAninhados.tipo_1.subtipo_B' : '$.dadosAninhados[?(@.tipo=="tipo 1")].subtipos[?(@.subtipo=="subtipo B")].valor',
    'dadosAninhados.tipo_2.subtipo_A' : '$.dadosAninhados[?(@.tipo=="tipo 2")].subtipos[?(@.subtipo=="subtipo A")].valor',
    'dadosAninhados.tipo_2.subtipo_B' : '$.dadosAninhados[?(@.tipo=="tipo 2")].subtipos[?(@.subtipo=="subtipo B")].valor',
    'campo_lista_dinamica_simples' : '$.campo_lista_dinamica_simples',
}

def run_json_query(json_payload, json_query):
    jsonpath_expr = parse(json_query)
    matches = jsonpath_expr.find(json_payload)
    return json.dumps(safely_get_first_value(matches))

def safely_get_first_value(your_list):
    first_match = next(iter(your_list or []), None)
    if first_match:
        return first_match.value


tabulated_data = {
    # key: json.dumps(parse(value).find(json_payload)[0].value)
    key: run_json_query(json_payload, value)
    for key, value
    in lookup_dict.items()
}

tabulated_data

# jsonpath_expr = parse('$.dadosCnpj[?(@.texto=="Data de início de atividades")].valor')

# matches = [match.value for match in jsonpath_expr.find(json)]

# print(matches)

{'autorizacao.anoCalendario': '"2021"',
 'autorizacao.avisoLegal': '"Lorem Ipsum Dolor Sit Amet"',
 'autorizacao.cnpjTitular': '"1234567891011"',
 'autorizacao.dataHoraRegistro': '"2022-12-28T23:56:02.65"',
 'autorizacao.destinatario': '"60701190000104"',
 'autorizacao.titular': '"05982110639"',
 'autorizacao.token': '"lorem ipsum dolor sit amet"',
 'dadosCnpj.data_de_início_de_atividades': '"20191013"',
 'dadosCnpj.capital_social': '"100000.00"',
 'dadosCnpj.data_de_exclusão_do_simples': 'null',
 'dadosAninhados.tipo_1.subtipo_A': '"valor_1A"',
 'dadosAninhados.tipo_1.subtipo_B': '"valor_1B"',
 'dadosAninhados.tipo_2.subtipo_A': '"valor_2A"',
 'dadosAninhados.tipo_2.subtipo_B': '"valor_2B"',
 'campo_lista_dinamica_simples': '["a", "b", "c"]'}

In [63]:
json_payload = {
    "foo": "bar",
    "nested": {
        "nested_key": "nested_value",
        "another_nested_key": "another_value",
        "deep_nested_obj": {
            "deep_nested_key_a": "deep_nested_value_a",
            "deep_nested_key_b": "deep_nested_value_b"
        }
    },
    "some_list": [
        "aaaa",
        "bbbb",
        "cccc"
    ],
    "complex_list": [
        {
            "key_a": "first_code",
            "key_b": "first_value",
            "key_c": "first_field_name"
        },
        {
            "key_a": "second_code",
            "key_b": "second_value",
            "key_c": "second_field_name"
        },
        {
            "key_a": "third_code",
            "key_b": "third_value",
            "key_c": "third_field_name"
        }
    ],
    "another_field": "lorem ipsum",
    "john": "doe",
    "lalala": {
        "papapa": "tatata",
        "nanana": "mamama"
    }
}

lookup_dict = {
    'foo': '$.foo',
    'nested.nested_key': '$.nested.nested_key',
    'nested.another_nested_key': '$.nested.another_nested_key',
    'nested.deep_nested_obj.deep_nested_key_a': '$.nested.deep_nested_obj.deep_nested_key_a',
    'nested.deep_nested_obj.deep_nested_key_b': '$.nested.deep_nested_obj.deep_nested_key_b',
    'some_list': '$.some_list',
    'complex_list.first_field_name': '$.complex_list[?(@.key_c=="first_field_name")].key_b',
    'complex_list.second_field_name': '$.complex_list[?(@.key_c=="second_field_name")].key_b',
    'complex_list.third_field_name': '$.complex_list[?(@.key_c=="third_field_name")].key_b',
    'another_field': '$.another_field',
    'john': '$.john',
    'lalala.papapa': '$.lalala.papapa',
    'lalala.nanana': '$.lalala.nanana',
}

def run_json_query(json_payload, json_query):
    jsonpath_expr = parse(json_query)
    matches = jsonpath_expr.find(json_payload)
    return json.dumps(safely_get_first_value(matches))

def safely_get_first_value(your_list):
    first_match = next(iter(your_list or []), None)
    if first_match:
        return first_match.value


tabulated_data = {
    # key: json.dumps(parse(value).find(json_payload)[0].value)
    key: run_json_query(json_payload, value)
    for key, value
    in lookup_dict.items()
}

tabulated_data

{'foo': '"bar"',
 'nested.nested_key': '"nested_value"',
 'nested.another_nested_key': '"another_value"',
 'nested.deep_nested_obj.deep_nested_key_a': '"deep_nested_value_a"',
 'nested.deep_nested_obj.deep_nested_key_b': '"deep_nested_value_b"',
 'some_list': '["aaaa", "bbbb", "cccc"]',
 'complex_list.first_field_name': '"first_value"',
 'complex_list.second_field_name': '"second_value"',
 'complex_list.third_field_name': '"third_value"',
 'another_field': '"lorem ipsum"',
 'john': '"doe"',
 'lalala.papapa': '"tatata"',
 'lalala.nanana': '"mamama"'}

In [61]:
import json

def generate_lookup_dict(data, parent_key='', sep='.'):
    items = {}
    for k, v in data.items():
        new_key = f"{parent_key}{sep}{k}" if parent_key else k
        if isinstance(v, dict):
            items.update(generate_lookup_dict(v, new_key, sep))
        elif isinstance(v, list):
            items[new_key] = f"$.{new_key}"
        else:
            items[new_key] = f"$.{new_key}"
    return items

# Example JSON data
json_data = """
{
    "foo": "bar",
    "nested": {
        "nested_key": "nested_value",
        "another_nested_key": "another_value",
        "deep_nested_obj": {
            "deep_nested_key_a": "deep_nested_value_a",
            "deep_nested_key_b": "deep_nested_value_b"
        }
    },
    "some_list": [
        "aaaa",
        "bbbb",
        "cccc"
    ],
    "complex_list": [
        {
            "key_a": "first_code",
            "key_b": "first_value",
            "key_c": "first_field_name"
        },
        {
            "key_a": "second_code",
            "key_b": "second_value",
            "key_c": "second_field_name"
        },
        {
            "key_a": "third_code",
            "key_b": "third_value",
            "key_c": "third_field_name"
        }
    ],
    "another_field": "lorem ipsum",
    "john": "doe",
    "lalala": {
        "papapa": "tatata",
        "nanana": "mamama"
    }
}
"""

# Convert the JSON data to a Python dictionary
json_payload = json.loads(json_data)

# Generate the lookup dictionary
lookup_dict = generate_lookup_dict(json_payload)

# Print the lookup dictionary
for k, v in lookup_dict.items():
    print(f"'{k}': '{v}',")
lookup_dict

def run_json_query(json_payload, json_query):
    jsonpath_expr = parse(json_query)
    matches = jsonpath_expr.find(json_payload)
    return json.dumps(safely_get_first_value(matches))

def safely_get_first_value(your_list):
    first_match = next(iter(your_list or []), None)
    if first_match:
        return first_match.value


tabulated_data = {
    # key: json.dumps(parse(value).find(json_payload)[0].value)
    key: run_json_query(json_payload, value)
    for key, value
    in lookup_dict.items()
}

print(tabulated_data)

'foo': '$.foo',
'nested.nested_key': '$.nested.nested_key',
'nested.another_nested_key': '$.nested.another_nested_key',
'nested.deep_nested_obj.deep_nested_key_a': '$.nested.deep_nested_obj.deep_nested_key_a',
'nested.deep_nested_obj.deep_nested_key_b': '$.nested.deep_nested_obj.deep_nested_key_b',
'some_list': '$.some_list',
'complex_list': '$.complex_list',
'another_field': '$.another_field',
'john': '$.john',
'lalala.papapa': '$.lalala.papapa',
'lalala.nanana': '$.lalala.nanana',
{'foo': '"bar"', 'nested.nested_key': '"nested_value"', 'nested.another_nested_key': '"another_value"', 'nested.deep_nested_obj.deep_nested_key_a': '"deep_nested_value_a"', 'nested.deep_nested_obj.deep_nested_key_b': '"deep_nested_value_b"', 'some_list': '["aaaa", "bbbb", "cccc"]', 'complex_list': '[{"key_a": "first_code", "key_b": "first_value", "key_c": "first_field_name"}, {"key_a": "second_code", "key_b": "second_value", "key_c": "second_field_name"}, {"key_a": "third_code", "key_b": "third_value", "key

In [59]:
import json

def generate_lookup_dict(data, parent_key='', sep='.'):
    items = {}
    for k, v in data.items():
        new_key = f"{parent_key}{sep}{k}" if parent_key else k
        if isinstance(v, dict):
            items.update(generate_lookup_dict(v, new_key, sep))
        elif isinstance(v, list):
            items[new_key] = f"$.{new_key}"
        else:
            items[new_key] = f"$.{new_key}"
    return items

# Example JSON data
json_data = """
{
    "foo": "bar",
    "nested": {
        "nested_key": "nested_value",
        "another_nested_key": "another_value",
        "deep_nested_obj": {
            "deep_nested_key_a": "deep_nested_value_a",
            "deep_nested_key_b": "deep_nested_value_b"
        }
    },
    "some_list": [
        "aaaa",
        "bbbb",
        "cccc"
    ],
    "complex_list": [
        {
            "key_a": "value_1",
            "key_b": "value_2",
            "key_c": "first_field_name"
        },
        {
            "key_a": "value_3",
            "key_b": "value_4",
            "key_c": "second_field_name"
        },
        {
            "key_a": "value_5",
            "key_b": "value_6",
            "key_c": "third_field_name"
        }
    ],
    "another_field": "lorem ipsum",
    "john": "doe",
    "lalala": {
        "papapa": "tatata",
        "nanana": "mamama"
    }
}
"""

# Convert the JSON data to a Python dictionary
json_payload = json.loads(json_data)

# Generate the lookup dictionary
lookup_dict = generate_lookup_dict(json_payload)

# Print the lookup dictionary
for k, v in lookup_dict.items():
    print(f"'{k}': '{v}',")
lookup_dict

def run_json_query(json_payload, json_query):
    jsonpath_expr = parse(json_query)
    matches = jsonpath_expr.find(json_payload)
    return json.dumps(safely_get_first_value(matches))

def safely_get_first_value(your_list):
    first_match = next(iter(your_list or []), None)
    if first_match:
        return first_match.value


tabulated_data = {
    # key: json.dumps(parse(value).find(json_payload)[0].value)
    key: run_json_query(json_payload, value)
    for key, value
    in lookup_dict.items()
}

print(tabulated_data)

'foo': '$.foo',
'nested.nested_key': '$.nested.nested_key',
'nested.another_nested_key': '$.nested.another_nested_key',
'nested.deep_nested_obj.deep_nested_key_a': '$.nested.deep_nested_obj.deep_nested_key_a',
'nested.deep_nested_obj.deep_nested_key_b': '$.nested.deep_nested_obj.deep_nested_key_b',
'some_list': '$.some_list',
'complex_list': '$.complex_list',
'another_field': '$.another_field',
'john': '$.john',
'lalala.papapa': '$.lalala.papapa',
'lalala.nanana': '$.lalala.nanana',
{'foo': '"bar"', 'nested.nested_key': '"nested_value"', 'nested.another_nested_key': '"another_value"', 'nested.deep_nested_obj.deep_nested_key_a': '"deep_nested_value_a"', 'nested.deep_nested_obj.deep_nested_key_b': '"deep_nested_value_b"', 'some_list': '["aaaa", "bbbb", "cccc"]', 'complex_list': '[{"key_a": "value_1", "key_b": "value_2", "key_c": "first_field_name"}, {"key_a": "value_3", "key_b": "value_4", "key_c": "second_field_name"}, {"key_a": "value_5", "key_b": "value_6", "key_c": "third_field_name"

In [68]:
import json

def generate_lookup_dict(data, parent_key='', sep='.', field_name=None, value_field=None):
    items = {}
    for k, v in data.items():
        new_key = f"{parent_key}{sep}{k}" if parent_key else k
        if isinstance(v, dict):
            items.update(generate_lookup_dict(v, new_key, sep, field_name, value_field))
        elif isinstance(v, list) and field_name and value_field:
            for item in v:
                if isinstance(item, dict) and item.get(field_name) is not None:
                    items[f"{new_key}{sep}{item[field_name]}"] = f"$.{new_key}[?(@.{field_name}=='{item[field_name]}')].{value_field}"
        else:
            items[new_key] = f"$.{new_key}"
    return items

# Example JSON data
json_data = """
{
    "autorizacao": {
        "anoCalendario": "2021",
        "avisoLegal": "Lorem Ipsum Dolor Sit Amet",
        "cnpjTitular": "1234567891011",
        "dataHoraRegistro": "2022-12-28T23:56:02.65",
        "destinatario": "60701190000104",
        "titular": "05982110639",
        "token": "lorem ipsum dolor sit amet"
    },
    "dadosCnpj": [
        {
            "codigo": "5.1",
            "texto": "Data de início de atividades",
            "valor": "20191013"
        },
        {
            "codigo": "5.2",
            "texto": "Capital Social",
            "valor": "100000.00"
        },
        {
            "codigo": "5.3",
            "texto": "Data de exclusão do simples",
            "valor": "null"
        }
    ],
    "dadosAninhados": [
        {
            "tipo": "tipo 1",
            "subtipos": [
                {
                    "subtipo": "subtipo A",
                    "campo": "campo_1A",
                    "valor": "valor_1A"
                },
                {
                    "subtipo": "subtipo B",
                    "campo": "campo_1B",
                    "valor": "valor_1B"
                }
            ]
        },
        {
            "tipo": "tipo 2",
            "subtipos": [
                {
                    "subtipo": "subtipo A",
                    "campo": "campo_2A",
                    "valor": "valor_2A"
                },
                {
                    "subtipo": "subtipo B",
                    "campo": "campo_2B",
                    "valor": "valor_2B"
                }
            ]
        }
    ],
    "campo_lista_dinamica_simples": ["a", "b", "c"],
    "campo_lista_dinamica_complexa": [
        {
            "nome": "fulano",
            "sobrenome": "silva"
        },
        {
            "nome": "fulana",
            "sobrenome": "santos"
        }
    ]
}
"""

# Convert the JSON data to a Python dictionary
json_payload = json.loads(json_data)

# Define the desired field name and value fields
field_name = "texto"
value_field = "valor"

# Generate the lookup dictionary
lookup_dict = generate_lookup_dict(json_payload, field_name=field_name, value_field=value_field)

# Print the lookup dictionary
for k, v in lookup_dict.items():
    print(f"'{k}': '{v}',")


'autorizacao.anoCalendario': '$.autorizacao.anoCalendario',
'autorizacao.avisoLegal': '$.autorizacao.avisoLegal',
'autorizacao.cnpjTitular': '$.autorizacao.cnpjTitular',
'autorizacao.dataHoraRegistro': '$.autorizacao.dataHoraRegistro',
'autorizacao.destinatario': '$.autorizacao.destinatario',
'autorizacao.titular': '$.autorizacao.titular',
'autorizacao.token': '$.autorizacao.token',
'dadosCnpj.Data de início de atividades': '$.dadosCnpj[?(@.texto=='Data de início de atividades')].valor',
'dadosCnpj.Capital Social': '$.dadosCnpj[?(@.texto=='Capital Social')].valor',
'dadosCnpj.Data de exclusão do simples': '$.dadosCnpj[?(@.texto=='Data de exclusão do simples')].valor',


In [80]:
import json

def generate_lookup_dict(data, parent_key='', sep='.', field_mappings=None):
    items = {}
    for k, v in data.items():
        new_key = f"{parent_key}{sep}{k}" if parent_key else k
        if isinstance(v, dict):
            items.update(generate_lookup_dict(v, new_key, sep, field_mappings))
        elif isinstance(v, list) and field_mappings and isinstance(field_mappings, dict) and field_mappings.get(new_key):
            for item in v:
                if isinstance(item, dict):
                    field_name = field_mappings[new_key]['field_name']
                    value_field = field_mappings[new_key]['value_field']
                    if field_name in item:
                        items[f"{new_key}.{item[field_name]}"] = f"$.{new_key}[?(@.{field_name}=='{item[field_name]}')].{value_field}"
        elif field_mappings and isinstance(field_mappings, dict) and field_mappings.get(new_key):
            field_name = field_mappings[new_key]['field_name']
            value_field = field_mappings[new_key]['value_field']
            items[new_key] = f"$.{new_key}[?(@.{field_name}=='{field_name}')].{value_field}"
        else:
            items[new_key] = f"$.{new_key}"
    return items

# Example JSON data
json_payload = {
    "autorizacao": {
        "anoCalendario": "2021",
        "avisoLegal": "Lorem Ipsum Dolor Sit Amet",
        "cnpjTitular": "1234567891011",
        "dataHoraRegistro": "2022-12-28T23:56:02.65",
        "destinatario": "60701190000104",
        "titular": "05982110639",
        "token": "lorem ipsum dolor sit amet"
    },
    "dadosCnpj": [
        {
            "codigo": "5.1",
            "texto": "Data de início de atividades",
            "valor": "20191013"
        },
        {
            "codigo": "5.2",
            "texto": "Capital Social",
            "valor": "100000.00"
        },
        {
            "codigo": "5.3",
            "texto": "Data de exclusão do simples",
            "valor": "null"
        }
    ],
    "teste": {
        "interno": [
            {
                "Codigo": "6.1",
                "Texto": "Campo de teste 6.1",
                "Valor": "AAAAAAAAAAAAAA"
            },
            {
                "Codigo": "6.2",
                "Texto": "Campo de teste 6.2",
                "Valor": "BBBBBBBBBBBBBBBB"
            }
        ]
    },
    "dadosAninhados": [
        {
            "tipo": "tipo 1",
            "subtipos": [
                {
                    "subtipo": "subtipo A",
                    "campo": "campo_1A",
                    "valor": "valor_1A"
                },
                {
                    "subtipo": "subtipo B",
                    "campo": "campo_1B",
                    "valor": "valor_1B"
                }
            ]
        },
        {
            "tipo": "tipo 2",
            "subtipos": [
                {
                    "subtipo": "subtipo A",
                    "campo": "campo_2A",
                    "valor": "valor_2A"
                },
                {
                    "subtipo": "subtipo B",
                    "campo": "campo_2B",
                    "valor": "valor_2B"
                }
            ]
        }
    ],
    "campo_lista_dinamica_simples": ["a", "b", "c"],
    "campo_lista_dinamica_complexa": [
        {
            "nome": "fulano",
            "sobrenome": "silva"
        },
        {
            "nome": "fulana",
            "sobrenome": "santos"
        }
    ]
}

# Define the field mappings
field_mappings = {
    "dadosCnpj": {'field_name': 'texto', 'value_field': 'valor'},
    "teste.interno": {'field_name': 'texto', 'value_field': 'valor'},
    "dadosAninhados.tipo_1.subtipos": {'field_name': 'subtipo', 'value_field': 'valor'},
    "dadosAninhados.tipo_2.subtipos": {'field_name': 'subtipo', 'value_field': 'valor'},
}

# Generate the lookup dictionary
lookup_dict = generate_lookup_dict(json_payload, field_mappings=field_mappings)

# Print the lookup dictionary
for k, v in lookup_dict.items():
    print(f"'{k}': '{v}',")


'autorizacao.anoCalendario': '$.autorizacao.anoCalendario',
'autorizacao.avisoLegal': '$.autorizacao.avisoLegal',
'autorizacao.cnpjTitular': '$.autorizacao.cnpjTitular',
'autorizacao.dataHoraRegistro': '$.autorizacao.dataHoraRegistro',
'autorizacao.destinatario': '$.autorizacao.destinatario',
'autorizacao.titular': '$.autorizacao.titular',
'autorizacao.token': '$.autorizacao.token',
'dadosCnpj.Data de início de atividades': '$.dadosCnpj[?(@.texto=='Data de início de atividades')].valor',
'dadosCnpj.Capital Social': '$.dadosCnpj[?(@.texto=='Capital Social')].valor',
'dadosCnpj.Data de exclusão do simples': '$.dadosCnpj[?(@.texto=='Data de exclusão do simples')].valor',
'dadosAninhados': '$.dadosAninhados',
'campo_lista_dinamica_simples': '$.campo_lista_dinamica_simples',
'campo_lista_dinamica_complexa': '$.campo_lista_dinamica_complexa',


In [73]:
lookup_dict = {
    'autorizacao.anoCalendario' : '$.autorizacao.anoCalendario',
    'autorizacao.avisoLegal' : '$.autorizacao.avisoLegal',
    'autorizacao.cnpjTitular' : '$.autorizacao.cnpjTitular',
    'autorizacao.dataHoraRegistro' : '$.autorizacao.dataHoraRegistro',
    'autorizacao.destinatario' : '$.autorizacao.destinatario',
    'autorizacao.titular' : '$.autorizacao.titular',
    'autorizacao.token' : '$.autorizacao.token',
    'dadosCnpj.data_de_início_de_atividades' : '$.dadosCnpj[?(@.codigo=="5.1")].valor',
    'dadosCnpj.capital_social' : '$.dadosCnpj[?(@.codigo=="5.2")].valor',
    'dadosCnpj.data_de_exclusão_do_simples' : '$.dadosCnpj[?(@.codigo=="5.3")].valor',
    'dadosCnpj.data_de_exclusão_do_simples' : '$.dadosCnpj[?(@.codigo=="5.4")].valor',
    'dadosAninhados.tipo_1.subtipo_A' : '$.dadosAninhados[?(@.tipo=="tipo 1")].subtipos[?(@.subtipo=="subtipo A")].valor',
    'dadosAninhados.tipo_1.subtipo_B' : '$.dadosAninhados[?(@.tipo=="tipo 1")].subtipos[?(@.subtipo=="subtipo B")].valor',
    'dadosAninhados.tipo_2.subtipo_A' : '$.dadosAninhados[?(@.tipo=="tipo 2")].subtipos[?(@.subtipo=="subtipo A")].valor',
    'dadosAninhados.tipo_2.subtipo_B' : '$.dadosAninhados[?(@.tipo=="tipo 2")].subtipos[?(@.subtipo=="subtipo B")].valor',
    'campo_lista_dinamica_simples' : '$.campo_lista_dinamica_simples',
}
lookup_dict

{'autorizacao.anoCalendario': '$.autorizacao.anoCalendario',
 'autorizacao.avisoLegal': '$.autorizacao.avisoLegal',
 'autorizacao.cnpjTitular': '$.autorizacao.cnpjTitular',
 'autorizacao.dataHoraRegistro': '$.autorizacao.dataHoraRegistro',
 'autorizacao.destinatario': '$.autorizacao.destinatario',
 'autorizacao.titular': '$.autorizacao.titular',
 'autorizacao.token': '$.autorizacao.token',
 'dadosCnpj.data_de_início_de_atividades': '$.dadosCnpj[?(@.codigo=="5.1")].valor',
 'dadosCnpj.capital_social': '$.dadosCnpj[?(@.codigo=="5.2")].valor',
 'dadosCnpj.data_de_exclusão_do_simples': '$.dadosCnpj[?(@.codigo=="5.4")].valor',
 'dadosAninhados.tipo_1.subtipo_A': '$.dadosAninhados[?(@.tipo=="tipo 1")].subtipos[?(@.subtipo=="subtipo A")].valor',
 'dadosAninhados.tipo_1.subtipo_B': '$.dadosAninhados[?(@.tipo=="tipo 1")].subtipos[?(@.subtipo=="subtipo B")].valor',
 'dadosAninhados.tipo_2.subtipo_A': '$.dadosAninhados[?(@.tipo=="tipo 2")].subtipos[?(@.subtipo=="subtipo A")].valor',
 'dadosAninha